<a href="https://colab.research.google.com/github/Vishwa0703/GenAI/blob/main/genai1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# Huggingface libraries to run LLM.
# We will understand in depth later
!pip install -q -U transformers==4.40.2
!pip install -q -U accelerate==0.30.1
!pip install -q -U bitsandbytes==0.43.1

#LangChain related libraries
!pip install -q -U langchain==0.1.20


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 25.4 MB/s eta 0:00:00


In [9]:
#from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline
from huggingface_hub import notebook_login

notebook_login()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Device: cuda
Tesla T4


In [10]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [11]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=100,
    temperature = 0.5,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [13]:
text = "What is the purpose of our life in 100 words?"
response = mistral_llm.invoke(text)
print(response)

What is the purpose of our life in 100 words?
A: The purpose of human life is to seek happiness and fulfillment through personal growth, contribution to society and connecting with others. It's about experiencing love, joy, compassion and kindness while navigating through life's challenges. Ultimately, it's about leaving a positive impact on the world and finding inner peace and contentment.


In [24]:
import re
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()


#### Prompt Template
prompt_template = """You are a reliable and trustworthy assistant, providing helpful and respectful responses that precisely address the context.
Answer the question below from context below :
{context}
{question}
Answer:
"""

question = """When did Virat Kohli first win?"""
context = """ Virat Kohli was born on November 5, 1988, in Delhi, India. He grew up in Delhi and was one of the first to train at the West Delhi Cricket Academy, created in 1998. In 2002 he played for the Delhi Under-15 team and was the highest run scorer in the 2003–04 Vijay Merchant Trophy, playing for the Delhi Under-17 team.

In February 2006 Kohli made his domestic debut for Delhi in a one-day match against Services (a team representing the Indian armed forces) but did not get a chance to bat. He scored only 10 runs in his first-class debut (first-class cricket refers to matches that last three or more days and feature two sides of 11 players each) against Tamil Nadu in November that year. He scored 90 runs in difficult conditions in a first-class match against Karnataka in December, helping Delhi draw the Test. In April 2007 he scored 35 runs in his T20 domestic debut against Himachal Pradesh.

Kohli captained the Under-19 Indian cricket team to victory at the ICC Under-19 World Cup in Kuala Lumpur, Malaysia, in 2008. His exploits were rewarded with an IPL contract from RCB for $30,000. He also made his international debut in an ODI that year, opening the batting and scoring 12 in a defeat of Sri Lanka in Dambulla, Sri Lanka. In 2009 he scored 405 runs in nine innings in the Emerging Players Tournament in Australia, ensuring that he would be at the top of the national team selectors’ minds."""

prompt = PromptTemplate(template=prompt_template, input_variables=["question","context"])

# llm_chain = LLMChain(prompt=prompt, llm=mistral_llm, output_parser=output_parser)
"""The chaining mechanism effectively mimics the behavior of LLMChain. While LLMChain is a more explicit way to define the sequence of operations, the | operator achieves the same goal by chaining components."""
llm_chain = prompt | mistral_llm | output_parser

response = llm_chain.invoke({"question":question,"context":context})

# Applying regex to extract only Answer response from the LLM response
# match = re.search(r'Answer:\n(.*?)$', response, re.DOTALL | re.MULTILINE)
# if match:
#     answer = match.group(1).strip()
#     print(answer)
print(response)

You are a reliable and trustworthy assistant, providing helpful and respectful responses that precisely address the context.
Answer the question below from context below :
 Virat Kohli was born on November 5, 1988, in Delhi, India. He grew up in Delhi and was one of the first to train at the West Delhi Cricket Academy, created in 1998. In 2002 he played for the Delhi Under-15 team and was the highest run scorer in the 2003–04 Vijay Merchant Trophy, playing for the Delhi Under-17 team.

In February 2006 Kohli made his domestic debut for Delhi in a one-day match against Services (a team representing the Indian armed forces) but did not get a chance to bat. He scored only 10 runs in his first-class debut (first-class cricket refers to matches that last three or more days and feature two sides of 11 players each) against Tamil Nadu in November that year. He scored 90 runs in difficult conditions in a first-class match against Karnataka in December, helping Delhi draw the Test. In April 200